## 건강관리 앱 리뷰 텍스트마이닝 기반 디자인 전략 연구1
### LDA 토픽 모델링과 동시출현 단어 네트워크 분석 기법을 중심으로
#### 👨‍💻 Author: Gyeongbin Park(a.k.a., Tony Park)
  - 📬 Contact: dev.gbpark@gmail.com

- 👨‍🔬 분석 방법론
  - LDA Topic Modeling
  - Word Co-occurrence Network Analysis(TBA)

- 🛠 개발환경
  - Python 3.8
    - KoNLPy Mecab
  - 가상환경: pipenv
  - Gephi(네트워크 시각화)

- 🗂 데이터셋
  - 국내 구글 플레이 스토어 내 건강관리 앱 리뷰 54만 건

- 📚 참고
  - Blog: https://heytech.tistory.com/
  - Github: https://github.com/park-gb

# 패키지 설치

1) 가상환경
- 가상환경 pipenv 사용 시 아래 명령어를 통해 모든 필요 패키지 설치 가능
    - pipenv install

2) KoNLPy Mecab 설치방법
- 명령어
    - bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
- 설치 오류 시 해결방법: https://heytech.tistory.com/395?category=453616

# 패키지 import

In [79]:
import pandas as pd
# 경고 메시지 무시
import warnings
warnings.filterwarnings(action='ignore')
# 한국어 형태소 분석기 중 성능이 가장 우수한 Mecab 사용
from konlpy.tag import Mecab
from tqdm import tqdm # 작업 프로세스 시각화
import re # 문자열 처리를 위한 정규표현식 패키지
from gensim import corpora # 단어 빈도수 계산 패키지
import gensim # LDA 모델 활용 목적
import pyLDAvis.gensim_models # LDA 시각화용 패키지

In [86]:
mecab = Mecab()

# 데이터셋 load

In [3]:
dataset_raw = pd.read_excel('./data/dataset_raw.xlsx')
dataset_raw.head()

,app,review,rating
0,다리 근육 운동 – 4주 프로그램,다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐,1
1,다리 근육 운동 – 4주 프로그램,굿,5
2,다리 근육 운동 – 4주 프로그램,최고입니디,5
3,다리 근육 운동 – 4주 프로그램,아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.😂,5
4,다리 근육 운동 – 4주 프로그램,ᆞ,5


# 데이터 탐색
- 데이터셋에서 전반적으로 결측치 존재여부, 데이터 타입, 데이터 개수를 확인합니다.

In [4]:
dataset_raw.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540076 entries, 0 to 540075
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   app     540076 non-null  object
 1   review  540074 non-null  object
 2   rating  540076 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 12.4+ MB


# 데이터 전처리

## 결측치 확인

In [5]:
dataset_raw.isnull().sum()

app       0
review    2
rating    0
dtype: int64

## 결측치 제거

In [6]:
# axis = 0: 결측치 포함한 모든 행 제거
dataset = dataset_raw.dropna(axis = 0)
dataset.isnull().sum()

app       0
review    0
rating    0
dtype: int64

## 전처리용 딕셔너리 Load

### 불용어 사전 리스트
- 사용자의 서비스 경험과는 무관하지만 빈출되는 단어인 불용어는 미리 제거하는 작업이 필요합니다.

In [58]:
stopword_list = pd.read_excel('./data/stopword_list.xlsx')
stopword_list.head()

,stopword
0,가까스로
1,가량
2,가령
3,가민
4,가민커넥트


### 데이터 치환 리스트
- 같은 의미로 사용된 여러 데이터를 특정 단어로 통일화하기 위한 데이터셋입니다.
- 본 프로젝트에서는 특정 단어의 빈출여부가 핵심이기 때문에 같은 의미의 단어를 통일화하는 게 중요합니다.

In [59]:
replace_list = pd.read_excel('./data/replace_list.xlsx')
replace_list.head()

,before_replacement,after_replacement
0,S헬스,삼성헬스
1,LG폰,스마트폰
2,LG V10,스마트폰
3,G7,스마트폰
4,GX,그룹운동


### 한 글자 키워드 리스트
- 일반적으로 토큰이 1글자인 경우는 대부분 불용어에 해당하기 때문에 제거합니다.
- 단, 프로젝트 성격에 따라 1글자의 단어가 핵심 키워드가 될 수 있습니다.
- 아래의 리스트는 이처럼 키워드로 생각되는 1글자 단어를 리스트업 하였습니다.

In [89]:
one_char_keyword = pd.read_excel('./data/one_char_list.xlsx')
one_char_keyword.head()

,one_char_keyword
0,컵
1,방
2,물
3,돈
4,꿈


## 데이터 치환

In [73]:
# 같은 의미로 사용된 데이터를 하나의 단어로 통일화하는 함수
def replace_word(review):
    for i in range(len(replace_list['before_replacement'])):
        try:
            # 치환할 단어가 있는 경우에만 데이터 치환 수행
            if replace_list['before_replacement'][i] in review:
                review = review.replace(replace_list['before_replacement'][i], replace_list['after_replacement'][i])
        except Exception as e:
            print(f"Error 발생 / 에러명: {e}")
    return review

In [74]:
dataset['review_replaced'] = ''
review_replaced_list = []
for review in tqdm(dataset['review']):
    review_replaced = replace_word(str(review)) # 문자열 데이터 변환
    replaced_review_list.append(review_replaced)
dataset['review_replaced'] = review_replaced_list
dataset.head()

100%|██████████████████████████████████| 540074/540074 [16:57<00:00, 530.67it/s]


,app,review,rating,review_replaced
0,다리 근육 운동 – 4주 프로그램,다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐,1,다른 P4P어플과 연동 하면 기존에 있던 스케줄이 싹 사라짐
1,다리 근육 운동 – 4주 프로그램,굿,5,굿
2,다리 근육 운동 – 4주 프로그램,최고입니디,5,최고입니디
3,다리 근육 운동 – 4주 프로그램,아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.😂,5,아무곳에서나 확인 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.😂
4,다리 근육 운동 – 4주 프로그램,ᆞ,5,ᆞ


## 한국어 외 텍스트 제거

In [84]:
review_removed = list(map(lambda review: re.sub('[^가-힣 ]', '', review), dataset['review_replaced']))
review_removed[:5]

['다른 어플과 연동 하면 기존에 있던 스케줄이 싹 사라짐',
 '굿',
 '최고입니디',
 '아무곳에서나 확인 억지로라도 운동할수 있어서 너무 좋습니다 감사히 잘 쓸께요',
 '']

## 토큰화

In [88]:
review_tokenized = list(map(lambda review: mecab.nouns(review), review_removed))
review_tokenized[:5]

[['플', '연동', '기존', '스케줄'], ['굿'], ['최고', '디'], ['곳', '확인', '운동', '수'], []]

## 불용어 제거

In [90]:
def remove_stopword(tokens):
    review_removed_stopword = []
    for token in tokens:
        if 1 < len(token):
            if token not in stopword_list:
                review_removed_stopword.append(token)
        else:
            if token in one_char_keyword:
                review_removed_stopword.append(token)
    return review_removed_stopword

In [114]:
review_removed_stopword = list(map(lambda tokens : remove_stopword(tokens), review_tokenized))
review_prep = []
for tokens in review_removed_stopword:
    if 1 < len(tokens):
        review_prep.append(tokens)
review_prep[:5]

[['연동', '기존', '스케줄'],
 ['확인', '운동'],
 ['완전', '유용', '사용', '효과', '최고', '감사', '추천'],
 ['다리전', '운동', '효과', '운동', '도움'],
 ['남자', '여자', '구분', '효과', '실용']]

# LDA 토픽 모델링

## 단어 인코딩 및 빈도수 계산
- (단어 인덱스, 빈도수) 형태 만들기

In [118]:
dictionary = corpora.Dictionary(review_prep)
corpus = [dictionary.doc2bow(review) for review in review_prep]

  0%|                                                | 0/540074 [24:44<?, ?it/s]


## LDA 모델 훈련

### 하이퍼파라미터 튜닝

In [120]:
NUM_TOPICS = 10 # 토픽 개수는 하이퍼파라미터
# passes: 딥러닝에서 Epoch와 같은 개념으로, 전체 corpus로 모델 학습 횟수 결정
PASSES = 15 

In [121]:
# LDA 모델 학습
model = gensim.models.ldamodel.LdaModel(corpus, 
                                        num_topics = NUM_TOPICS, 
                                        id2word = dictionary, 
                                        passes = PASSES)

# LDA 토픽 모델링 결과 시각화

In [122]:
# 주피터 노트북 내 시각화
pyLDAvis.enable_notebook()
result_visualized = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
pyLDAvis.display(result_visualized)

In [123]:
# 시각화 자료 로컬 내 저장
RESULT_FILE = './result/lda_result.html'
pyLDAvis.save_html(result_visualized, RESULT_FILE)

/Users/gbpark/.local/share/virtualenvs/mHealthapp-review-analysis1-rzGFRNeI/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/gbpark/.local/share/virtualenvs/mHealthapp-review-analysis1-rzGFRNeI/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/gbpark/.local/share/virtualenvs/mHealthapp-review-analysis1-rzGFRNeI/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/gbpark/.local/share/virtualenvs/mHealthapp-review-analysis1-rzGFRNeI/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. 

/Users/gbpark/.local/share/virtualenvs/mHealthapp-review-analysis1-rzGFRNeI/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/gbpark/.local/share/virtualenvs/mHealthapp-review-analysis1-rzGFRNeI/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/gbpark/.local/share/virtualenvs/mHealthapp-review-analysis1-rzGFRNeI/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/gbpark/.local/share/virtualenvs/mHealthapp-review-analysis1-rzGFRNeI/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. 